# **1. Encryption Without Password**

In [ ]:
import cv2
import numpy as np

def encrypt_message(image_path, message, output_path):
    img = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
    if img is None:
        print("Error: Image not found!")
        return

    msg_len = len(message)
    if msg_len > img.shape[0] * img.shape[1]:
        print("Error: Message is too long for this image!")
        return

    message = str(msg_len).zfill(5) + message  # Store length in first 5 chars
    ascii_values = [ord(c) for c in message]

    index = 0
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            if index < len(ascii_values):
                img[i, j, 0] = ascii_values[index]  # Store in Blue channel
                index += 1
            else:
                break

    cv2.imwrite(output_path, img)
    print(f"Message encrypted successfully! Saved as {output_path}")

if __name__ == "__main__":
    image_path = input("Enter image path: ")
    message = input("Enter secret message: ")
    output_path = "encrypted_image.png"
    encrypt_message(image_path, message, output_path)


# **2. Decryption Without Password**

In [41]:

import cv2

def decrypt_message(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
    if img is None:
        print("Error: Image not found!")
        return

    ascii_values = []
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            ascii_values.append(img[i, j, 0])

    msg_length = int("".join(chr(ascii_values[i]) for i in range(5)))  # First 5 chars = length
    message = "".join(chr(ascii_values[i]) for i in range(5, 5 + msg_length))

    print("\nDecrypted Message:", message)

if __name__ == "__main__":
    image_path = input("Enter encrypted image path: ")
    decrypt_message(image_path)


Enter encrypted image path: /content/encrypted_image.png

Decrypted Message: Bt3A7EJO5VEQaMk0Wih1dQyq6btrqG/MLoAY6wcFp+k=


# **3. Encryption using a Password**

*   Enter Image Path
*   Enter Your Secret Message
*   Enter Password
*   You will get your Encrypted image as encrypted_image.png







In [39]:
import cv2
import hashlib
import numpy as np
from Crypto.Cipher import AES
import base64

def pad_message(message):
    return message + (16 - len(message) % 16) * chr(16 - len(message) % 16)

def encrypt_aes(message, password):
    key = hashlib.sha256(password.encode()).digest()
    cipher = AES.new(key, AES.MODE_ECB)
    encrypted_msg = cipher.encrypt(pad_message(message).encode())
    return base64.b64encode(encrypted_msg).decode()

def encrypt_message(image_path, message, password, output_path):
    img = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
    if img is None:
        print("Error: Image not found!")
        return

    encrypted_msg = encrypt_aes(message, password)
    msg_len = len(encrypted_msg)

    if msg_len > img.shape[0] * img.shape[1]:
        print("Error: Message is too long for this image!")
        return

    # Store message length in first 5 pixels
    encrypted_msg = str(msg_len).zfill(5) + encrypted_msg
    ascii_values = [ord(c) for c in encrypted_msg]

    index = 0
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            if index < len(ascii_values):
                img[i, j, 0] = ascii_values[index]  # Store in Blue channel
                index += 1
            else:
                break

    cv2.imwrite(output_path, img)
    print(f"Message encrypted successfully! Saved as {output_path}")

if __name__ == "__main__":
    image_path = input("Enter image path: ")
    message = input("Enter secret message: ")
    password = input("Set a password: ")
    output_path = "encrypted_image.png"
    encrypt_message(image_path, message, password, output_path)


Enter image path: /content/mypic.jpg
Enter secret message: Hi, This is Amit !
Set a password: Amit@007
Message encrypted successfully! Saved as encrypted_image.png


# **4. Decryption using a Password**

*   Enter Image Path
*   Enter Password
*   You will get your decrypted message

In [40]:
import cv2
import hashlib
import base64
from Crypto.Cipher import AES

def unpad_message(message):
    return message[:-ord(message[-1])]

def decrypt_aes(encrypted_msg, password):
    key = hashlib.sha256(password.encode()).digest()
    cipher = AES.new(key, AES.MODE_ECB)
    decrypted_msg = cipher.decrypt(base64.b64decode(encrypted_msg)).decode()
    return unpad_message(decrypted_msg)

def decrypt_message(image_path, password):
    img = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
    if img is None:
        print("Error: Image not found!")
        return

    ascii_values = []
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            ascii_values.append(img[i, j, 0])

    msg_length = int("".join(chr(ascii_values[i]) for i in range(5)))  # First 5 chars = length
    encrypted_msg = "".join(chr(ascii_values[i]) for i in range(5, 5 + msg_length))

    try:
        decrypted_msg = decrypt_aes(encrypted_msg, password)
        print("\nDecrypted Message:", decrypted_msg)
    except:
        print("Error: Incorrect password or corrupted data!")

if __name__ == "__main__":
    image_path = input("Enter encrypted image path: ")
    password = input("Enter password for decryption: ")
    decrypt_message(image_path, password)


Enter encrypted image path: /content/encrypted_image.png
Enter password for decryption: Amit@007

Decrypted Message: Hi, This is Amit !


# **5. Find the secret Message directly From the Encrypted Text**

In [42]:
import base64
import hashlib
from Crypto.Cipher import AES

def decrypt_aes(encrypted_msg, password):
    key = hashlib.sha256(password.encode()).digest()  # Generate 256-bit key
    cipher = AES.new(key, AES.MODE_ECB)  # ECB mode
    decrypted_msg = cipher.decrypt(base64.b64decode(encrypted_msg))

    # Remove padding
    pad_length = decrypted_msg[-1]
    decrypted_msg = decrypted_msg[:-pad_length]

    return decrypted_msg.decode()

# Encrypted text (Base64 format)
encrypted_text = "Bt3A7EJO5VEQaMk0Wih1dQyq6btrqG/MLoAY6wcFp+k="
password = "Amit@007"  # Your password

try:
    decrypted_message = decrypt_aes(encrypted_text, password)
    print("Decrypted message:", decrypted_message)
except Exception as e:
    print("Decryption failed:", e)


Decrypted message: Hi, This is Amit !


# **This is Testing Purpose Only**

In [19]:
import cv2
import os
import string

img = cv2.imread("mypic.jpg") # Replace with the correct image path

msg = input("Enter secret message:")
password = input("Enter a passcode:")

d = {}
c = {}

for i in range(255):
    d[chr(i)] = i
    c[i] = chr(i)

m = 0
n = 0
z = 0

for i in range(len(msg)):
    img[n, m, z] = d[msg[i]]
    n = n + 1
    m = m + 1
    z = (z + 1) % 3

cv2.imwrite("encryptedImage.jpg", img)
os.system("start encryptedImage.jpg")  # Use 'start' to open the image on Windows

message = ""
n = 0
m = 0
z = 0

pas = input("Enter passcode for Decryption")
if password == pas:
    for i in range(len(msg)):
        message = message + c[img[n, m, z]]
        n = n + 1
        m = m + 1
        z = (z + 1) % 3
    print("Decryption message:", message)
else:
    print("YOU ARE NOT auth")

Enter secret message:KKM
Enter a passcode:1111
Enter passcode for Decryption11
YOU ARE NOT auth
